In [1]:
import cv2
import numpy as np
from collections import Counter
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from scipy.stats import mode

In [2]:
# Function to get the dominant color
def get_dominant_color(image):
    pixels = image.reshape(-1, image.shape[-1])
    dominant_color = mode(pixels, axis=0, keepdims=True).mode.flatten()
    return dominant_color

In [19]:
# Function to get the eye color
def get_eye_color(dominant_color):
    color_ranges = {
        'black': ([10, 10, 10], [40, 40, 40]),
        'blue': ([90, 0, 0], [255, 95, 95]),
        'green': ([0, 90, 0], [95, 255, 95]),
        'brown': ([0, 0, 90], [95, 95, 255]),
        'hazel': ([40, 50, 0], [180, 160, 50]),
        'amber': ([40, 60, 80], [180, 150, 120]),
        'gray': ([40, 40, 40], [180, 180, 180]),
        'violet': ([100, 0, 100], [180, 95, 255]),
        'red': ([120, 0, 0], [255, 40, 40])
    }

    for color, (lower, upper) in color_ranges.items():
        for color_val in dominant_color:
            if np.all(color_val >= lower) and np.all(color_val <= upper):
                return color

    return 'black'

In [13]:
def select_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        detect_eye_color(file_path)
    else:
        messagebox.showinfo("Error", "No image selected.")

In [14]:
# Function to detect eye color
def detect_eye_color(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    if len(eyes) > 0:
        (x, y, w, h) = eyes[0]
        eye_region = gray[y:y+h, x:x+w]
        eye_dominant_color = get_dominant_color(eye_region)
        eye_color = get_eye_color(eye_dominant_color)
        result_label.config(text="Detected eye color: " + eye_color)
    else:
        result_label.config(text="No eyes detected.")

In [15]:
# Create the Tkinter GUI
window = tk.Tk()
window.title("Eye Color Detection")
window.geometry("400x300")

''

In [16]:
# Create a button to select an image
select_button = tk.Button(window, text="Select Image", command=select_image)
select_button.pack(pady=20)

In [17]:
# Create a label to display the result
result_label = tk.Label(window, text="Detected eye color:")
result_label.pack()

In [18]:
# Run the Tkinter event loop
window.mainloop()